In [1]:
import os
import json
import shutil
import glob
import torch
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
import logging
import pandas as pd

from ultralytics import YOLO

In [2]:
# Check if the MPS (Metal Performance Shaders) backend is available for Apple Silicon devices
if torch.backends.mps.is_available():
    device = "mps"
# If MPS is not available, check if CUDA (NVIDIA GPUs) is available
elif torch.cuda.is_available():
    device = "cuda"
# If neither MPS nor CUDA is available, fall back to using the CPU
else:
    device = "cpu"
# Print the device being used for training
print(f"🚀 Using device: {device}")

🚀 Using device: mps


In [3]:
META_JSON_FILE = 'meta.json'
with open(META_JSON_FILE, "r") as meta_file:
    meta_data = json.load(meta_file)

class_names = [
    "longitudinal crack",
    "transverse crack",
    "alligator crack",
    "block crack",
    "pothole",
    "manhole cover",
    "other corruption"
]
class_map = {name: idx for idx, name in enumerate(class_names)}
print("Loaded class mapping:", class_map)

Loaded class mapping: {'longitudinal crack': 0, 'transverse crack': 1, 'alligator crack': 2, 'block crack': 3, 'pothole': 4, 'manhole cover': 5, 'other corruption': 6}


In [4]:
# Step 1: Creating Data_Yaml File for training the model
DATA_YAML_PATH = "data.yaml"
data_yaml_content = f"""
path: ./datasets/
train: yolo_dataset/train/images
val: yolo_dataset/val/images
nc: {len(class_map)}
names: {class_names}
"""

with open(DATA_YAML_PATH, "w") as f:
    f.write(data_yaml_content)

print("data.yaml created successfully!")

data.yaml created successfully!


In [9]:
DATA_YAML_PATH = "data.yaml"

In [11]:
# Step 2: Load the YOLO model with pre-trained weights
model = YOLO("runs/detect/train/weights/last.pt") # Using YOLOv11m as the base model
model.to(device)
results = model.train(
    data=DATA_YAML_PATH,
    epochs=150,
    imgsz=640,
    batch=16,
    device=device,
    workers=0,
    patience=30,
    optimizer="SGD",
    lr0=0.01,
    lrf=0.1,
    weight_decay=0.0005,
    plots=True,
    save=True,
    val=True,
    augment=True, 
    amp=True,
    resume=True
)

print("Training Completed!")

New https://pypi.org/project/ultralytics/8.3.213 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.203 🚀 Python-3.10.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20011MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/train/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64

AssertionError: runs/detect/train/weights/last.pt training to 150 epochs is finished, nothing to resume.
Start a new training without resuming, i.e. 'yolo train model=runs/detect/train/weights/last.pt'

In [13]:
model = YOLO("yolo11s.pt") #Using YOLOv11s as the base model
model.load("runs/detect/train/weights/best.pt")
model.to(device)
results = model.train(
    data=DATA_YAML_PATH,
    epochs=60,
    imgsz=640,
    batch=16,
    device=device,
    workers=0,
    patience=30,
    optimizer="SGD",
    dropout=0.2,
    mosaic=0.7,
    lr0=0.001,
    plots=True,
    save=True,
    val=True,
    pretrained=True,
    amp=True,

)
print("Fine Tuning Complete!!!!")

Transferred 280/499 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.213 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.203 🚀 Python-3.10.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20011MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=0.7, multi_sca

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



      15/60      4.96G       1.49       1.58      1.377         40        640: 100% ━━━━━━━━━━━━ 2857/2857 2.8it/s 17:15<0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 240/240 2.2it/s 1:48<0.2s
                   all       7677      12025      0.615      0.548      0.576      0.311

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      16/60      4.96G       1.49      1.571      1.375         45        640: 100% ━━━━━━━━━━━━ 2857/2857 2.8it/s 17:15<0.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 240/240 2.2it/s 1:48<0.2s
                   all       7677      12025      0.623      0.543       0.58      0.314

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      17/60      4.96G      1.486      1.548      1.369         22        640: 100% ━━━━━━━━━━━━ 2857/2857 2.8it/s 17:11<0.3s
                 C

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



      19/60      4.96G      1.469      1.525      1.363         29        640: 100% ━━━━━━━━━━━━ 2857/2857 2.8it/s 17:14<0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 240/240 2.2it/s 1:48<0.2s
                   all       7677      12025      0.632       0.55      0.585      0.318

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      20/60      4.96G      1.463      1.514      1.357         52        640: 75% ━━━━━━━━━─── 2147/2857 2.6it/s 12:58<4:32

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



      43/60      4.96G      1.346      1.278      1.278         32        640: 100% ━━━━━━━━━━━━ 2857/2857 2.8it/s 17:14<0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 240/240 2.2it/s 1:47<0.2s
                   all       7677      12025      0.648      0.566      0.603       0.33

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      44/60      4.96G      1.338      1.264      1.271         51        640: 52% ━━━━━━────── 1492/2857 2.8it/s 8:60<8:077

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



      46/60      4.96G      1.336      1.254      1.272         33        640: 100% ━━━━━━━━━━━━ 2857/2857 2.8it/s 17:14<0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 240/240 2.2it/s 1:47<0.2s
                   all       7677      12025      0.648      0.567      0.604      0.331

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      47/60      4.96G      1.331      1.246      1.268         29        640: 100% ━━━━━━━━━━━━ 2857/2857 2.8it/s 17:14<0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 240/240 2.2it/s 1:47<0.2s
                   all       7677      12025      0.648      0.567      0.605      0.331

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      51/60      4.96G      1.295      1.119      1.268         12        640: 100% ━━━━━━━━━━━━ 2857/2857 2.8it/s 16:54<0.4s
                 C

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

